# Dashboard MVP prototype using Panel

In [ ]:
# import modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.preprocessing import StandardScaler
from ta.volatility import BollingerBands
from ta.trend import MACD, SMAIndicator
from ta.momentum import RSIIndicator
import panel as pn
import hvplot.pandas

In [ ]:
# Load dataset
preprocessed_df = pd.read_feather('../outputs/nb2_market_index_with_historical_and_features.feather')

In [ ]:
# --- Prepare the DataFrame ---
model_df = preprocessed_df.dropna().copy().reset_index().rename(columns={"index": "Date"})
model_df["Date"] = pd.to_datetime(model_df["Date"])

# --- Define Features & Target ---
feature_cols = ["Has_Event", "SMA_4", "RSI_4", "MACD", "MACD_Signal", "BB_Width"]
X = model_df[feature_cols]
y = model_df["Sentiment_Label"]

# --- Time-based Train/Test Split ---
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
import xgboost as xgb

split_index = int(len(model_df) * 0.8)
train_df = model_df.iloc[:split_index]
test_df = model_df.iloc[split_index:]

X_train = train_df[feature_cols]
y_train = train_df["Sentiment_Label"]
X_test = test_df[feature_cols]
y_test = test_df["Sentiment_Label"]

# --- Scale Features ---
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# --- Train Model ---
model = xgb.XGBClassifier(
    objective="binary:logistic",
    n_estimators=100,
    learning_rate=0.1,
    max_depth=3,
    random_state=42
)
model.fit(X_train_scaled, y_train)

# --- Predict on Test Set Only ---
y_pred = model.predict(X_test_scaled)



# --- Add Predictions to model_df (only for test part) ---
model_df["Predicted_Label"] = np.nan  # initialize with NaNs
model_df.loc[split_index:, "Predicted_Label"] = y_pred

# Predict probabilities (confidence that label is bullish)
y_proba = model.predict_proba(X_test_scaled)[:, 1]  # Probability of class 1 (bullish)

# Store in the test portion of model_df
model_df.loc[split_index:, "Confidence_Score"] = y_proba

# Mark correctness
model_df["Correct"] = np.nan
model_df.loc[split_index:, "Correct"] = (
    model_df.loc[split_index:, "Predicted_Label"] == model_df.loc[split_index:, "Sentiment_Label"]
)


# --- Accuracy & Classification Report ---
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
accuracy_text = f"""<span style="color:green; font-weight:bold; font-size:16px;">
✅ Model Accuracy (on unseen data): {accuracy:.2%}
</span>"""

# report = classification_report(y_test, y_pred, target_names=["Bearish", "Bullish"])

# Get the most recent week of features
X_next = preprocessed_df.dropna().copy().tail(1)[feature_cols]
X_next_scaled = scaler.transform(X_next)

# Predict next week
next_label = model.predict(X_next_scaled)[0]
next_proba = model.predict_proba(X_next_scaled)[0][1]  # Probability of bullish

# Format
label_str = "📈 Bullish" if next_label == 1 else "📉 Bearish"
confidence_str = f"{next_proba:.2%}"

# --- PANEL DASHBOARD ---
import panel as pn
import hvplot.pandas

pn.extension('plotly')

# Widgets
date_slider = pn.widgets.DateRangeSlider(
    name='Date Range',
    start=model_df["Date"].min(),
    end=model_df["Date"].max(),
    value=(model_df["Date"].min(), model_df["Date"].max())
)

# Interactive plot
@pn.depends(date_slider.param.value)
def sentiment_plot(date_range):
    start = pd.to_datetime(date_range[0])
    end = pd.to_datetime(date_range[1])
    df = model_df[(model_df["Date"] >= start) & (model_df["Date"] <= end)]

    # Base line plot
    plot = df.hvplot.line(x="Date", y="Lithium_Market_Index", label="Lithium Index", line_width=2, color="dodgerblue")

    # Actual sentiment markers
    bull = df[df["Sentiment_Label"] == 1].hvplot.scatter(
        x="Date", y="Lithium_Market_Index", color="green", size=40, marker="^", label="Bullish")
    bear = df[df["Sentiment_Label"] == 0].hvplot.scatter(
        x="Date", y="Lithium_Market_Index", color="red", size=40, marker="v", label="Bearish")

    # Predicted bullish markers (only in test portion)
    pred_bull = df[df["Predicted_Label"] == 1].hvplot.scatter(
        x="Date", y="Lithium_Market_Index", color="blue", marker="*", size=100, label="Predicted Bullish"
    )
    
    # Incorrect predictions (predicted bullish, actually bearish)
    incorrect = df[(df["Predicted_Label"] == 1) & (df["Correct"] == False)].hvplot.scatter(
        x="Date", y="Lithium_Market_Index", color="black", marker="x", size=100, label="Incorrect Prediction")
    

    return plot * bull * bear * pred_bull * incorrect

# Markdown & text panes
accuracy_display = pn.pane.Markdown(accuracy_text, sizing_mode="stretch_width")
# report_display = pn.pane.Markdown(report, name="Classification Report")

forecast_card = pn.pane.Markdown(
    f"""
### 🔮 **Next Week Forecast**
- **Sentiment:** {label_str}
- **Confidence (of being Bullish):** {confidence_str}
""",
    sizing_mode="stretch_width"
)

# Final layout
dashboard = pn.Column(
    pn.pane.Markdown("## 🔋 Lithium Market Sentiment Dashboard"),
    accuracy_display,
    date_slider,
    sentiment_plot,    
    forecast_card
)

dashboard.show()


## Next week prediction

In [ ]:
# 1. Grab the last known week
latest = returns_df.dropna().copy().tail(1)

# 2. Build the same feature columns
X_next = latest[feature_cols]

# 3. Scale it using the existing scaler
X_next_scaled = scaler.transform(X_next)

# 4. Predict sentiment and confidence
next_label = model.predict(X_next_scaled)[0]
next_proba = model.predict_proba(X_next_scaled)[0][1]  # Confidence bullish

# 5. Show result
label_str = "Bullish" if next_label == 1 else "Bearish"
confidence_pct = f"{next_proba:.2%}"

print(f"📅 Next Week Sentiment Prediction: **{label_str}**")
print(f"🔍 Confidence (for being Bullish): {confidence_pct}")
